In [1]:
import plotly.graph_objs as go
import plotly.offline as pyo
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_renderer
import pandas as pd
import os
import pandas as pd
import seaborn as sns
sns.set()
import numpy as np
import matplotlib.pyplot as plt

cash_outlet = pd.read_csv('training.csv')

cash_outlet.head()

,ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,...,Q17,Q18,Q19,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification
0,5086,98,2,3,1,1,2,2,0,0,...,-1,4,4,-4.460442,29.811396,0,0,0,0,0
1,1258,40,1,1,3,5,1,1,1,0,...,4,1,4,-6.176438,39.244871,1,1,1,0,3
2,331,18,2,4,6,3,2,1,0,0,...,-1,1,1,-6.825702,37.652798,1,0,0,0,2
3,6729,50,1,1,3,1,1,1,0,0,...,-1,1,4,-3.372049,35.808307,1,0,1,0,3
4,8671,34,1,1,1,1,2,1,0,1,...,-1,1,4,-7.179645,31.039095,1,1,0,1,3


In [2]:
cash_outlet.rename(inplace=True, columns={
    'Q1':'age',
    'Q2':'gender',
    'Q3':'marital status',
    'Q4':'Highest level of education',
    'Q5':'property ownership',
    'Q6':'land ownership',
    'Q7':'personal mobile status',
    'Q8_1':'salaries or wages',
    'Q8_2':'money trading or selling',
    'Q8_3':'money providing a service',
    'Q8_4':'money casual work',
    'Q8_5':'money rental income',
    'Q8_6':'money interests',
    'Q8_7':'money pension',
    'Q8_8':'money social welfare',
    'Q8_9':'money financially dependant',
    'Q8_10':'money no income',
    'Q8_11':'other',
    'Q9':'salary and wages',
    'Q10':'selling things',
    'Q11':'providing a service',
    'Q12':'send money to another country',
    'Q13':'money last sent',
    'Q14':'received money from another country',
    'Q15':'last money received',
    'Q16':'mobile money purchases',
    'Q17':'mobile money bills',
    'Q18':'literacy in kwiswhahili',
    'Q19':'literacy in english'
    
})

In [3]:
cash_outlet.gender= ['male' if each == 1 else 'female' for each in cash_outlet.gender]
cash_outlet.groupby(by='gender').size()

gender
female    3972
male      3122
dtype: int64

In [4]:
cash_outlet.groupby(by='mobile_money_classification').size()

mobile_money_classification
0    1386
1    1778
2     804
3    3126
dtype: int64

In [5]:
cash_outlet['mobile_money_classification'].replace([0,1,2,3],['Nun','FS_only','MM_only','MM_FS'],inplace=True)

In [6]:
cash_outlet.groupby(by='mobile_money_classification').size()

mobile_money_classification
FS_only    1778
MM_FS      3126
MM_only     804
Nun        1386
dtype: int64

In [7]:
cash_outlet['Group by age'] = cash_outlet.age.apply(lambda x : int(x/25) * 10)

In [8]:
cash_outlet['marital status'].replace([1,2,3,4],['Married','Divorced','Widowed','Never married'],inplace=True)

In [9]:
cash_outlet.groupby(by='marital status').size()

marital status
Divorced          687
Married          4548
Never married    1181
Widowed           678
dtype: int64

In [10]:
cash_outlet['land ownership'].replace([1,2],['Yes', 'No'],inplace=True)

In [11]:
cash_outlet.groupby(by='land ownership').size()

land ownership
No     5963
Yes    1131
dtype: int64

In [12]:
female=cash_outlet[cash_outlet['gender']==2]
male=cash_outlet[cash_outlet['gender']==1]

m_class=cash_outlet['mobile_money_classification']

Nun=cash_outlet[cash_outlet['mobile_money_classification']==0]
FS_only=cash_outlet[cash_outlet['mobile_money_classification']==1]
MM_only=cash_outlet[cash_outlet['mobile_money_classification']==2]
MM_FS=cash_outlet[cash_outlet['mobile_money_classification']==3]

Owner=cash_outlet[cash_outlet['land ownership'] == 1]
Non_owner=cash_outlet[cash_outlet['land ownership'] == 2]

married=cash_outlet[cash_outlet['mobile_money_classification']==1]
divorced=cash_outlet[cash_outlet['mobile_money_classification']==2]
widowed=cash_outlet[cash_outlet['mobile_money_classification']==3]
never=cash_outlet[cash_outlet['mobile_money_classification']==4]

In [13]:
app = dash.Dash()

# Feedback

I think the main problem are the dataframe to are referring too. For example the above defined
dataframes (female, male) are empty/blank so calling them won't produce anything.

In [14]:
male

,ID,age,gender,marital status,Highest level of education,property ownership,land ownership,personal mobile status,salaries or wages,money trading or selling,...,literacy in kwiswhahili,literacy in english,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification,Group by age


In [15]:
female

,ID,age,gender,marital status,Highest level of education,property ownership,land ownership,personal mobile status,salaries or wages,money trading or selling,...,literacy in kwiswhahili,literacy in english,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification,Group by age


Also another thing I notice is you are providing the whole dataframe instead of specified column or values.

Will try to plot the first figure you have and hopefully that will work and you'll use it as an template or
chart to refer too to plot other charts

# Example

In [16]:
# FigureOne dataset (Data to be used to produce a the first chart)
figA  = pd.DataFrame(m_class.value_counts()).reset_index()

figA.head()

,index,mobile_money_classification
0,MM_FS,3126
1,FS_only,1778
2,Nun,1386
3,MM_only,804


In [17]:
app.title = 'dashboard'


app.layout = html.Div(
       html.Div([
            html.H1(children = 'dashoard for mobile money'),
            html.Div(children ='''Dash: Web dashboards with python,'''),
        
            # Hopefull the below will serve as a guide for the rest of your charts
            dcc.Graph(id='mobile money',
                figure={
                    'data':[
                        {'x':figA["mobile_money_classification"], 'y':figA["index"], 'type': 'bar','name': 'Male'}
                           ],
                'layout':{'title':'Relationship between mobile money class and gender'}
                }
            )
        ]
       )
)


In [18]:
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Mar/2020 13:15:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 13:15:03] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_3m1580396678.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 13:15:03] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_2_3m1580396678.7.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 13:15:03] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_3m1580396678.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 13:15:03] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_2_3m1580396678.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 13:15:03] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_7_0m1580396406.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 13:15:03] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_7_0m1580396406.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 13:15

Output

![](output.png)